Plot the locations of the profiles for the different regions. 

In [1]:
import os
import glob
import netCDF4
import logging
import calendar
import numpy as np
import emodnetchemistry
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.patches import Polygon
coast = cfeature.GSHHSFeature(scale="i")
datacrs = ccrs.PlateCarree()
plt.style.use("calanus.mplstyle")
from importlib import reload
reload(emodnetchemistry)

<module 'emodnetchemistry' from '/home/ctroupin/Projects/EMODnet-Chemistry/python/emodnetchemistry.py'>

In [3]:
logger = logging.getLogger("EMODnet-Chemistry-Data-positions")
logger.setLevel(logging.DEBUG)
logging.info("Starting")

## Paths

In [5]:
#datadir = "/data/EMODnet/Eutrophication/Split/"
datadir = ""/home/ctroupin/data/EMODnet/2022""
datadirTS = os.path.join(datadir, "timeseries/")
figdir = "../figures/EMODnet2022/"

if not os.path.exists(figdir):
    os.makedirs(figdir)

varlist = ["phosphate", "silicate", "ammonium", "chlorophyll-a", 
           "dissolved_inorganic_nitrogen", "dissolved_oxygen"]

logger.debug("Already exists") if os.path.isdir(datadir) else logger.warning("Data directory does not exist")
logger.debug("Already exists") if os.path.isdir(figdir) else os.path.makedirs("figdir")

DEBUG:EMODnet-Chemistry-Data-positions:Already exists
DEBUG:EMODnet-Chemistry-Data-positions:Already exists


### Domains
Taken from another work, coordinates should be adapted.

In [6]:
domain = [-45., 70., 24., 83.]

arctic = emodnetchemistry.Region(-44.25, 70.0, 56.5, 83.0, 0.25, 0.125, "Arctic Sea")
neatlantic = emodnetchemistry.Region(-42.0, -0.1, 24.9, 48.0, 0.1, 0.1, "North-East Atlantic Ocean")
balticsea = emodnetchemistry.Region(9.4, 30.9, 53.0, 65.9, 0.1, 0.1, "Baltic Sea")
blacksea = emodnetchemistry.Region(26.5, 41.95, 40.0, 47.95, 0.05, 0.05, "Black Sea")
mediterranean = emodnetchemistry.Region(-7.0, 36.375, 30.0, 45.875, 0.125, 0.125, "Mediterranean Sea")
northsea = emodnetchemistry.Region(-5.4, 13.0, 47.9, 62.0, 0.1, 0.1, "North Sea")
regionlist = [arctic, blacksea, balticsea, mediterranean, neatlantic, northsea]

In [7]:
regiondict = {"ArcticSea": arctic, "NorthAtlantic": neatlantic, "BalticSea": balticsea, 
              "BlackSea": blacksea, "MediterraneanSea": mediterranean, "NorthSea": northsea}

## Make plot
Define projections: one global (Orthographic) and 2 local (Mercator and Lambert Azimuthal Equal Area) for testing purpose (should use the same for the paper).

In [8]:
globalproj = ccrs.Orthographic(central_longitude=5.0, central_latitude=50.0, globe=None)

localproj = ccrs.Mercator(central_longitude=0.5 * (domain[0] + domain[1]),
    min_latitude=domain[2],
    max_latitude=domain[3],
    globe=None)

localproj2 = ccrs.LambertAzimuthalEqualArea(central_longitude=0.5 * (domain[0] + domain[1]),
                                            central_latitude=0.5 * (domain[2] + domain[3]), 
                                      false_easting=0.0, 
                                      false_northing=0.0, globe=None)
localproj2

<Derived Projected CRS: +proj=laea +ellps=WGS84 +lon_0=12.5 +lat_0=53.5 +x ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Lambert Azimuthal Equal Area
Datum: Unknown based on WGS84 ellipsoid
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Loop on variables
### Profiles

In [9]:
reload(emodnetchemistry)
for varname in varlist:
    logger.info("Working on variable {}".format(varname))

    figname = os.path.join(figdir, f"EMODnet_positions_{varname}.png")
    logger.info("Saving figure as {}".format(figname))
    emodnetchemistry.plot_data_locations(globalproj, datadir, varname, regiondict, figname, bathy=False)
    
    figname = os.path.join(figdir, f"EMODnet_hexbin_{varname}.png")
    logger.info("Saving figure as {}".format(figname))
    emodnetchemistry.plot_hexbin_datalocations(globalproj, datadir, varname, figname)

INFO:EMODnet-Chemistry-Data-positions:Working on variable phosphate
INFO:EMODnet-Chemistry-Data-positions:Saving figure as ../figures/EMODnet2022/EMODnet_positions_phosphate.png
INFO:EMODnet-Chemistry-Data-positions:Data files in /home/ctroupin/data/EMODnet/2022
INFO:EMODnet-Chemistry-Data-positions:Working on 6 files
INFO:EMODnet-Chemistry-Data-positions:['/home/ctroupin/data/EMODnet/2022/ArcticSea_Water_body_phosphate.nc', '/home/ctroupin/data/EMODnet/2022/BalticSea_Water_body_phosphate.nc', '/home/ctroupin/data/EMODnet/2022/BlackSea_Water_body_phosphate.nc', '/home/ctroupin/data/EMODnet/2022/MediterraneanSea_Water_body_phosphate.nc', '/home/ctroupin/data/EMODnet/2022/NorthAtlantic_Water_body_phosphate.nc', '/home/ctroupin/data/EMODnet/2022/NorthSea_Water_body_phosphate.nc']
INFO:EMODnet-Chemistry-Data-positions:ArcticSea
INFO:EMODnet-Chemistry-Data-positions:BalticSea
INFO:EMODnet-Chemistry-Data-positions:BlackSea
INFO:EMODnet-Chemistry-Data-positions:MediterraneanSea
INFO:EMODnet-C

### Time series

In [28]:
reload(emodnetchemistry)
for varname in varlist:
    logger.info("Working on variable {}".format(varname))

    figname = os.path.join(figdir, f"EMODnet_positions_timeseries_{varname}.png")
    logger.info("Saving figure as {}".format(figname))
    emodnetchemistry.plot_data_locations(localproj, datadirTS, varname, regiondict, figname, 
                                         bathy=False, isglobal=False)

INFO:EMODnet-Chemistry-Data-positions:Working on variable phosphate
INFO:EMODnet-Chemistry-Data-positions:Saving figure as ../figures/EMODnet2022/EMODnet_positions_timeseries_phosphate.png
INFO:EMODnet-Chemistry-Data-positions:Data files in /data/EMODnet/Eutrophication/Split/timeseries/
INFO:EMODnet-Chemistry-Data-positions:Working on 5 files
INFO:EMODnet-Chemistry-Data-positions:['/data/EMODnet/Eutrophication/Split/timeseries/BalticSea_TS_Water_body_phosphate.nc', '/data/EMODnet/Eutrophication/Split/timeseries/BlackSea_TS_Water_body_phosphate.nc', '/data/EMODnet/Eutrophication/Split/timeseries/MediterraneanSea_TS_Water_body_phosphate.nc', '/data/EMODnet/Eutrophication/Split/timeseries/NorthAtlantic_TS_Water_body_phosphate.nc', '/data/EMODnet/Eutrophication/Split/timeseries/NorthSea_TS_Water_body_phosphate.nc']
INFO:EMODnet-Chemistry-Data-positions:BalticSea
INFO:EMODnet-Chemistry-Data-positions:BlackSea
INFO:EMODnet-Chemistry-Data-positions:MediterraneanSea
INFO:EMODnet-Chemistry-Data

### Data distribution month by month

In [22]:
for varname in varlist:
    datafilelist = sorted(glob.glob(os.path.join(datadir, f"*{varname}*.nc")))
    lonall, latall, datesall = emodnetchemistry.all_positions(datafilelist)
    years = [dd.year for dd in datesall]
    months = [dd.month for dd in datesall]

    months = np.array(months)
    years = np.array(years)
    fig = plt.figure(figsize=(12, 12))
    for mm in range(1, 13):
        mmm = str(mm).zfill(2)
        logger.info("Working on month {}".format(calendar.month_name[mm]))
        timeselector = np.where(months == mm)[0]
        logger.info("Found {} data points".format(len(timeselector)))

        ax = plt.subplot(4, 3, mm, projection=localproj2)
        ax.set_extent(domain, crs=datacrs)
        plt.title(calendar.month_name[mm])
        ax.plot(lonall[timeselector], latall[timeselector], "ko", ms=.02, transform=datacrs)
        ax.add_feature(coast, facecolor=".75", edgecolor=None)

    plt.savefig(os.path.join(figdir, f"data_{varname}_monthly_laea"))
    #plt.show()
    fig.clf()

INFO:EMODnet-Chemistry-Data-positions:Working on month January
INFO:EMODnet-Chemistry-Data-positions:Found 175505 data points
INFO:EMODnet-Chemistry-Data-positions:Working on month February
INFO:EMODnet-Chemistry-Data-positions:Found 161686 data points
INFO:EMODnet-Chemistry-Data-positions:Working on month March
INFO:EMODnet-Chemistry-Data-positions:Found 171589 data points
INFO:EMODnet-Chemistry-Data-positions:Working on month April
INFO:EMODnet-Chemistry-Data-positions:Found 241010 data points
INFO:EMODnet-Chemistry-Data-positions:Working on month May
INFO:EMODnet-Chemistry-Data-positions:Found 247640 data points
INFO:EMODnet-Chemistry-Data-positions:Working on month June
INFO:EMODnet-Chemistry-Data-positions:Found 257290 data points
INFO:EMODnet-Chemistry-Data-positions:Working on month July
INFO:EMODnet-Chemistry-Data-positions:Found 212360 data points
INFO:EMODnet-Chemistry-Data-positions:Working on month August
INFO:EMODnet-Chemistry-Data-positions:Found 245821 data points
INFO:E

INFO:EMODnet-Chemistry-Data-positions:Working on month July
INFO:EMODnet-Chemistry-Data-positions:Found 3751789 data points
INFO:EMODnet-Chemistry-Data-positions:Working on month August
INFO:EMODnet-Chemistry-Data-positions:Found 3045360 data points
INFO:EMODnet-Chemistry-Data-positions:Working on month September
INFO:EMODnet-Chemistry-Data-positions:Found 2779555 data points
INFO:EMODnet-Chemistry-Data-positions:Working on month October
INFO:EMODnet-Chemistry-Data-positions:Found 2546537 data points
INFO:EMODnet-Chemistry-Data-positions:Working on month November
INFO:EMODnet-Chemistry-Data-positions:Found 1702041 data points
INFO:EMODnet-Chemistry-Data-positions:Working on month December
INFO:EMODnet-Chemistry-Data-positions:Found 1115020 data points


<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

## Scatter plots of values

In [ ]:
datafile1 = "/home/ctroupin/data/EMODnet/2022/"